## **Import the pandas package for data preprocessing**

In [1]:
# import the pandas library for data the data cleaning
import pandas as pd
import re

## **Load the data**

In [2]:
# We load the entire excel file with all its sheet
data_path = "/home/student/Desktop/INDABAX_Challenge/Blood-campain-INDABAX-CHALLLENGE/data/Updated Challenge dataset.xlsx"
blood_campain = pd.read_excel(data_path, sheet_name=None)

#### **We check the name of the different sheet imported**

In [3]:
# we check the different sheet in the file that we have import.
print(blood_campain.keys())

dict_keys(['Candidat au don 2019 (avec anne', 'Donneurs 2019', 'Candidat au don 2019  (avec age'])


#### **We store the into different variable to be able to handle them eazily**

In [4]:
# extract the different sheets
df_with_Bthd = blood_campain["Candidat au don 2019 (avec anne"]
df_with_age = blood_campain["Candidat au don 2019  (avec age"]
donnors = blood_campain['Donneurs 2019']

##### **We get a look at the different shape of each sheet**

In [5]:
# shape of the different sheet
for key in blood_campain.keys():
    print(f"dimension of the sheet \" {key} \" {blood_campain[key].shape}")

dimension of the sheet " Candidat au don 2019 (avec anne " (1915, 39)
dimension of the sheet " Donneurs 2019 " (1900, 6)
dimension of the sheet " Candidat au don 2019  (avec age " (1846, 40)


### **We focus our interest to the sheet of candidate which has 1915 value for the preprocessing since the donnor has 1900 observation shich are more than the size of the second sheet**

### **We start our preprocessing by the Date variable**

#### **We write a function to handle the formatage of the date were the month come before the day and were there are no value**

In [6]:
def preprocess_date(date_str):
    try:
        # we attempt to convert to datetime
        date = pd.to_datetime(date_str)
    except ValueError:
        # If conversion fails, we assume the date is in the format 'm/d/yyyy'
        parts = date_str.split('/') # we split dates using / as separator 
        if len(parts) == 3 and int(parts[2]) < 1000: 
            # change the year in the good format with the year at 2019
            date_str = f"{parts[1]}/{parts[0]}/2019"
        date = pd.to_datetime(date_str)
    return date

#### **We store here all the date variable to avoid repetition in code application**

In [7]:
dates_var = ['Date de remplissage de la fiche','Date de naissance',"Si oui preciser la date du dernier don."]

#### **We apply our preprocess function to each date varible in the list**

In [8]:
for date in dates_var:
    df_with_Bthd[date] = df_with_Bthd[date].apply(preprocess_date)
    print(f"The column {date} is now of type {df_with_Bthd[date].dtype}")

/tmp/ipykernel_256074/2094283300.py:11: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  date = pd.to_datetime(date_str)


The column Date de remplissage de la fiche is now of type datetime64[ns]
The column Date de naissance is now of type datetime64[ns]
The column Si oui preciser la date du dernier don. is now of type datetime64[ns]


#### **We look now at the missing value and range of value for each column to ensure the reality to be reflected** 

In [9]:
# check the summary table to see the range of date and if there are missing values
df_with_Bthd[["Date de remplissage de la fiche", "Date de naissance", "Si oui preciser la date du dernier don."]].describe()

,Date de remplissage de la fiche,Date de naissance,Si oui preciser la date du dernier don.
count,1913,1915,798
mean,2019-05-10 12:38:00.815473152,1988-11-09 14:50:19.112271616,2016-04-02 19:43:45.563909888
min,1977-12-25 00:00:00,1882-11-18 00:00:00,1987-09-02 00:00:00
25%,2019-03-27 00:00:00,1983-04-16 12:00:00,2015-03-02 00:00:00
50%,2019-07-23 00:00:00,1989-09-30 00:00:00,2017-12-29 12:00:00
75%,2019-10-07 00:00:00,1994-11-07 12:00:00,2018-11-03 18:00:00
max,2020-10-12 00:00:00,2092-03-19 00:00:00,2020-10-06 00:00:00


#### **we can see that there are some errors to correct:**
- ##### there are two missing value in the fill date
- ##### the range of value in the fill date is not good for data collected in 2019
- ##### there is some one born in 2092
- ##### there are people with previous donnation in 2020 

#### **We handle the first point:**
- #### Missing values in the Fill date

In [10]:
# we found first the observation with missing values in the fill date
missing_filter = pd.isna(df_with_Bthd["Date de remplissage de la fiche"])
missing = df_with_Bthd[missing_filter]

# Take a look at their eligibility
missing[["Date de remplissage de la fiche","Date de naissance","Si oui preciser la date du dernier don.","ÉLIGIBILITÉ AU DON."]]

,Date de remplissage de la fiche,Date de naissance,Si oui preciser la date du dernier don.,ÉLIGIBILITÉ AU DON.
1548,NaT,1990-03-22,NaT,Définitivement non-eligible
1549,NaT,2020-11-23,2020-02-11,Eligible


#### **Looking at the obtained values and since the collect was in 2019 we assume that the two candidate fill the form the same date as the observation who next them and we assume also tha the oservation has the mediane age of candidate**

In [11]:
# we get the date of the observation that comes after them (index 1550)
next_fill_date = df_with_Bthd.loc[1550, "Date de remplissage de la fiche"]
# then we replace the dates of the 1548th and 1549th observations
df_with_Bthd.loc[1548:1549, "Date de remplissage de la fiche"] = next_fill_date
# set the buirth date of the 1549th observation
df_with_Bthd.loc[1549, "Date de naissance"] = pd.to_datetime("1989-09-30 00:00:00")

# Check again for missing
missing_filter = pd.isna(df_with_Bthd["Date de remplissage de la fiche"])
df_with_Bthd[missing_filter]

,Date de remplissage de la fiche,Date de naissance,Niveau d'etude,Genre,Taille,Poids,Situation Matrimoniale (SM),Profession,Arrondissement de résidence,Quartier de Résidence,...,"Raison de non-eligibilité totale [Porteur(HIV,hbs,hcv)]",Raison de non-eligibilité totale [Opéré],Raison de non-eligibilité totale [Drepanocytaire],Raison de non-eligibilité totale [Diabétique],Raison de non-eligibilité totale [Hypertendus],Raison de non-eligibilité totale [Asthmatiques],Raison de non-eligibilité totale [Cardiaque],Raison de non-eligibilité totale [Tatoué],Raison de non-eligibilité totale [Scarifié],Si autres raison préciser


#### **Now we look at fill date which are not with the year 2019**

In [12]:
# handle those who haven't fill the form in 2019
filter_outlier_2019 = df_with_Bthd["Date de remplissage de la fiche"].dt.year != 2019
out_2019 = df_with_Bthd[filter_outlier_2019]
# show
out_2019[["Date de remplissage de la fiche","Si oui preciser la date du dernier don.","ÉLIGIBILITÉ AU DON."]]

,Date de remplissage de la fiche,Si oui preciser la date du dernier don.,ÉLIGIBILITÉ AU DON.
2,1977-12-25,2009-01-01,Eligible
3,1986-02-06,NaT,Eligible
4,1988-05-20,NaT,Eligible
5,1988-05-20,NaT,Eligible
6,1990-02-07,2018-08-07,Eligible
7,1992-12-06,2018-02-01,Eligible
8,1995-01-09,2018-03-17,Eligible
9,2013-01-30,NaT,Eligible
10,2015-05-07,NaT,Eligible
11,2018-01-17,NaT,Eligible


In [13]:
# we now assume that all those who have fille the "Date de remplissage.." with year at 2020 wanted to fill 2019
def adjust_year(date):
    if ((date.year > 2019) or ((date.year > 2010) and (date.year < 2019))) :
        return date.replace(year=2019)
    return date

# apply the adjustment function
df_with_Bthd['Date de remplissage de la fiche'] = df_with_Bthd['Date de remplissage de la fiche'].apply(adjust_year)

# we check again 
filter_outlier_2019 = df_with_Bthd["Date de remplissage de la fiche"].dt.year != 2019
out_2019 = df_with_Bthd[filter_outlier_2019]
out_2019[["Date de remplissage de la fiche","Date de naissance","Si oui preciser la date du dernier don.","ÉLIGIBILITÉ AU DON."]]

,Date de remplissage de la fiche,Date de naissance,Si oui preciser la date du dernier don.,ÉLIGIBILITÉ AU DON.
2,1977-12-25,1977-12-25,2009-01-01,Eligible
3,1986-02-06,1986-02-06,NaT,Eligible
4,1988-05-20,1988-05-20,NaT,Eligible
5,1988-05-20,1988-05-20,NaT,Eligible
6,1990-02-07,1990-02-01,2018-08-07,Eligible
7,1992-12-06,1992-12-06,2018-02-01,Eligible
8,1995-01-09,1995-05-11,2018-03-17,Eligible
1595,1989-10-31,1989-11-19,NaT,Temporairement Non-eligible
1857,1994-05-30,1994-05-20,2016-01-01,Définitivement non-eligible


##### **we see that they have fill date "Date de remplissage.." with their birthdate our approach to handle that is to set as filling date the date of the nearlier quartile to avoid set to them the mediane value and increase the proportion people near to the second quartile which do not take in to account that the integer id may reflect an order of arrived of the candidates**

In [14]:
# Store the values of the first and third quartile
replacement_value = ["2019-03-27 00:00:00", "2019-10-07 00:00:00"]

# Replace values for indices 2 to 8
for i in range(2, 9):
    df_with_Bthd.loc[i, "Date de remplissage de la fiche"] = pd.to_datetime(replacement_value[0])

# Replace values for specific indices
df_with_Bthd.loc[1595, "Date de remplissage de la fiche"] = pd.to_datetime(replacement_value[1])
df_with_Bthd.loc[1857, "Date de remplissage de la fiche"] = pd.to_datetime(replacement_value[1])

In [15]:
# we check again 
filter_outlier_2019 = df_with_Bthd["Date de remplissage de la fiche"].dt.year != 2019
out_2019 = df_with_Bthd[filter_outlier_2019]
out_2019[["Date de remplissage de la fiche","Date de naissance","Si oui preciser la date du dernier don.","ÉLIGIBILITÉ AU DON."]]

,Date de remplissage de la fiche,Date de naissance,Si oui preciser la date du dernier don.,ÉLIGIBILITÉ AU DON.


#### **We now handle Birthdate mater**

In [16]:
# select the distinct year values in "Date de naissance"
distinct_years = df_with_Bthd["Date de naissance"].dt.year.unique()
print(distinct_years)

[1999 1978 1977 1986 1988 1990 1992 1995 1980 1991 1994 1975 1985 1993
 1998 1979 1981 1989 1982 1984 1997 1987 1972 1974 1996 1966 2000 1983
 2019 1971 1968 2020 1976 2089 2092 1973 1962 1969 1963 2001 1965 1970
 1964 2002 2008 2009 1967 1961 2007 1882 1957 2003 1960 1959 1958]


##### **According to the "World Health Organization (WHO)'', the regulatory age for being eligible to donate blood is generally between "18 and 65 years". However, some countries may allow donations from 16 or 17-year-olds with parental consent.**

In [17]:
# Calculate age in 2019
age_2019 = 2019 - df_with_Bthd["Date de naissance"].dt.year
# Select those older than 65 or younger than 16
out_WHDrangeover = age_2019 > 65
out_WHDrangeunder = age_2019 < 16
# Use bitwise OR (|) instead of 'or'
out_WHDobs = df_with_Bthd[out_WHDrangeover | out_WHDrangeunder]
# Display relevant columns
out_WHDobs[["Date de remplissage de la fiche", "Date de naissance", 
            "Si oui preciser la date du dernier don.", "ÉLIGIBILITÉ AU DON."]]


,Date de remplissage de la fiche,Date de naissance,Si oui preciser la date du dernier don.,ÉLIGIBILITÉ AU DON.
85,2019-01-16,2019-01-16,NaT,Eligible
106,2019-01-18,2020-03-19,2004-03-19,Eligible
112,2019-01-18,2020-03-20,2018-03-19,Eligible
114,2019-01-18,2089-06-10,NaT,Eligible
129,2019-01-21,2092-03-19,NaT,Eligible
132,2019-01-21,2020-05-04,NaT,Eligible
231,2019-02-06,2019-11-25,2018-10-06,Eligible
345,2019-03-03,2019-04-03,NaT,Eligible
375,2019-03-08,2019-06-06,2015-01-01,Eligible
380,2019-03-11,2019-07-16,2018-01-01,Eligible


In [18]:
# we asume that people with year "2089, 2092, " wanted to put 1989 and 1992 so we change their year
def fix_birthdate_year(date):
    if date.year > 2019:
        if date.year == 2089:
            date = date.replace(year=1989)
        elif date.year == 2092:
            date = date.replace(year=1992)
    elif date.year == 1882:
        date = date.replace(year=1992)
            
    return date

df_with_Bthd["Date de naissance"] = df_with_Bthd["Date de naissance"].apply(fix_birthdate_year)

In [19]:
size = df_with_Bthd.shape[0]
quarts = [int(size / 4), int(size / 2), int(size * 3 / 4)]
replacement_values = [
    "1983-04-26 00:00:00",
    "1989-09-30 00:00:00",
    "1994-10-23 12:00:00",
]

for index in out_WHDobs.index:
    if index < quarts[0]:  # First quartile
        df_with_Bthd.loc[index, "Date de naissance"] = pd.to_datetime(replacement_values[0])
    elif quarts[0] <= index and index < quarts[1]:  # Second quartile
        df_with_Bthd.loc[index, "Date de naissance"] = pd.to_datetime(replacement_values[1])
    elif quarts[1] <= index and index < quarts[2]:  # Third quartile
        df_with_Bthd.loc[index, "Date de naissance"] = pd.to_datetime(replacement_values[2])
    else:  # Fourth quartile (index >= quarts[2])
        df_with_Bthd.loc[index, "Date de naissance"] = pd.to_datetime(replacement_values[2])


#### **Last points on date, there are people with previous donnation in 2020**

In [20]:
df_with_Bthd["Si oui preciser la date du dernier don."].dt.year.unique()

array([2017., 2019., 2009.,   nan, 2018., 2020., 2011., 2014., 2002.,
       1999., 2013., 2005., 2016., 2004., 2015., 2008., 2006., 2010.,
       2012., 1987., 1990., 2000., 2007., 2003., 2001.])

In [21]:
# we just remove those who their previous donation is after 2019 since we don't have mean to infer this date
filter_wrong_previous = df_with_Bthd["Si oui preciser la date du dernier don."].dt.year > 2019
wrong_previous = df_with_Bthd[filter_wrong_previous]
wrong_previous[["Date de remplissage de la fiche", "Date de naissance", 
            "Si oui preciser la date du dernier don.", "ÉLIGIBILITÉ AU DON."]]

,Date de remplissage de la fiche,Date de naissance,Si oui preciser la date du dernier don.,ÉLIGIBILITÉ AU DON.
14,2019-04-01,1985-03-08,2020-02-29,Eligible
230,2019-02-06,1984-04-23,2020-10-06,Eligible
232,2019-02-06,1984-07-22,2020-09-06,Eligible
469,2019-04-01,1987-11-23,2020-02-29,Eligible
471,2019-04-01,1987-06-12,2020-02-29,Eligible
474,2019-04-01,1996-01-30,2020-02-29,Eligible
475,2019-04-01,2000-07-25,2020-02-29,Eligible
477,2019-04-01,2000-04-30,2020-02-29,Eligible
479,2019-04-02,1998-05-05,2020-02-29,Eligible
481,2019-04-02,1988-03-31,2020-02-29,Eligible


In [22]:
# we update "Si oui preciser la date du dernier don." by removing 3 months to "Date de remplissage de la fiche" since there are all eligible
for index in wrong_previous.index:
    df_with_Bthd.loc[index,"Si oui preciser la date du dernier don."] = df_with_Bthd.loc[index,"Date de remplissage de la fiche"] + pd.DateOffset(months=-3)

### **We jump now into the other variable**

In [23]:
# We first start by passing all the character value in the dataset to lower case
df_with_Bthd = df_with_Bthd.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_256074/3118645155.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_with_Bthd = df_with_Bthd.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [24]:
df_with_Bthd["Niveau d\'etude"].unique()

array(['pas précisé', 'universitaire', 'secondaire', 'primaire', 'aucun'],
      dtype=object)

In [25]:
df_with_Bthd["Genre"].unique()

array(['homme', 'femme'], dtype=object)

In [26]:
# drop poids and taille
df_with_Bthd = df_with_Bthd.drop(columns=["Poids", "Taille"])

In [27]:
df_with_Bthd["Situation Matrimoniale (SM)"].unique()

array(['célibataire', 'marié (e)', 'divorcé(e)', 'veuf (veuve)'],
      dtype=object)

In [28]:
df_with_Bthd["Arrondissement de résidence"].unique()

array(['douala 3', 'douala (non précisé )', 'douala 5', 'douala 1',
       'yaounde', 'douala 2', 'douala 4', 'bafoussam', 'dschang',
       'non précisé', 'buea', 'kribi', 'pas précisé', 'njombe', 'tiko',
       'edea', 'manjo', 'west', 'douala', 'yaoundé', 'oyack', 'nkouabang',
       'deido', 'pas mentionné', 'douala 6', 'batie', 'bomono ba mbegue',
       'meiganga', 'pas precise', 'non precisé', 'sud ouest tombel',
       'pas précisé ', 'ngodi bakoko ', 'ras', 'r a s ', 'limbe ',
       'r a s', 'dcankongmondo', 'boko', 'ras '], dtype=object)

In [29]:
arrondissement_dict = {
    "bafoussam": ["bafoussam"],
    "batie": ["batie"],
    "boko": ["boko"],
    "bomono ba mbegue": ["bomono ba mbengue"],
    "buea": ["buea", "buea", "ambiance"],
    "deido": ["deido"],
    "douala (non précisé )": ["douala", "douala"],
    "douala 1": ["deido", "bessengue"],
    "douala 2": ["new bell", "newbell", "newbell", "new bell", "ngodi bakoko"],
    "douala 3": ["logbaba", "nyalla", "nkolbong", "dakar","ndogpassi 2","pk8"],
    "douala 4": ["ndobo"],
    "douala 5": ["ngangue", "bepanda", "pk14", "makepe", "pk12", "sic cacao","logbessou "],
    "douala 6": ["bonaberi", "bonaberi", "bonaberi","bonaberie"],
    "dschang": ["dschang"],
    "edea": ["edea", "edea", "edea", "nkong mondo", "edea", "edea"],
    "kribi": ["kribi", "kribi"],
    "limbe": ["limbe"],
    "manjo": ["manjo"],
    "meiganga": ["meiganga"],
    "ngodi bakoko": ["ngodi bakoko"],
    "nkouabang": ["nkouabang"],
    "oyack": ["oyack"],
    "sud ouest tombel": ["tombel"],
    "tiko": ["tiko"],
    "west": ["west"],
    "yaounde": ["bastos", "mbanga", "nkolmesseng", "yaoundé", "damas"],
    "pas précisé": ["rien","non précisé", "aucun", "non précisé", "non précisé","pas mentionné","pas precise", "meiganga", "pas precise", "pas precisé", "pas précisé", "pas précisé", "pas précisé", "pas précisé", "pas précisé", "ras", "ras", "ras", "ras", "ras", "ras", "ras", "r a s", "r a s", "r a s", "soboum", "r a s", "r a s", "r a s", "r a s", "congo", "r a s"]
}

In [30]:
def clean_quartier(quartier):
    quartier = str(quartier).strip().lower()  # Normaliser le texte
    # Cas spécifique : "douala douala" devient "douala"
    if quartier == "douala douala":
        return "douala"
    # Supprimer "douala" en début ou fin de chaîne
    return re.sub(r'^(douala\s+|\s+douala$)', '', quartier)

# Appliquer la fonction sur la colonne "Quartier de Résidence"
df_with_Bthd["Quartier de Résidence"] = df_with_Bthd["Quartier de Résidence"].apply(clean_quartier)

In [31]:

def assign_arrondissement(df, keyword, arrondissement):
    """
    Assigns the given arrondissement to all rows where 'Quartier de Résidence' contains the keyword.
    
    Parameters:
    df (pd.DataFrame): The dataset containing the columns 'Quartier de Résidence' and 'Arrondissement de résidence'.
    keyword (str): The keyword to search for in 'Quartier de Résidence'.
    arrondissement (str): The arrondissement to assign when the keyword is found.
    
    Returns:
    pd.DataFrame: Updated DataFrame with arrondissement assigned.
    """
    df.loc[
        df["Quartier de Résidence"].str.contains(keyword, case=False, na=False),
        "Arrondissement de résidence"
    ] = arrondissement
    return df

for arrondissement, quartiers in arrondissement_dict.items():
    for quartier in quartiers:
        assign_arrondissement(df_with_Bthd,quartier,arrondissement)

In [32]:
df_with_Bthd

,Date de remplissage de la fiche,Date de naissance,Niveau d'etude,Genre,Situation Matrimoniale (SM),Profession,Arrondissement de résidence,Quartier de Résidence,Nationalité,Religion,...,"Raison de non-eligibilité totale [Porteur(HIV,hbs,hcv)]",Raison de non-eligibilité totale [Opéré],Raison de non-eligibilité totale [Drepanocytaire],Raison de non-eligibilité totale [Diabétique],Raison de non-eligibilité totale [Hypertendus],Raison de non-eligibilité totale [Asthmatiques],Raison de non-eligibilité totale [Cardiaque],Raison de non-eligibilité totale [Tatoué],Raison de non-eligibilité totale [Scarifié],Si autres raison préciser
0,2019-11-03,1999-09-01,pas précisé,homme,célibataire,chaudronnier,douala 3,logbaba,camerounaise,pas précisé,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-03-21,1978-02-04,universitaire,homme,marié (e),secrétaire comptable,douala 3,ndogpassi 2,camerounaise,chretien (ne de nouveau),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-03-27,1977-12-25,secondaire,femme,marié (e),commerçant (e),douala 3,dakar,camerounaise,chretien (catholique),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-03-27,1986-02-06,secondaire,homme,célibataire,homme d’affaire,douala 5,ngangue,camerounaise,pas précisé,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-03-27,1988-05-20,pas précisé,homme,célibataire,beat maker,douala (non précisé ),douala,camerounaise,chretien (catholique),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,2019-05-06,1982-03-06,universitaire,homme,célibataire,r a s,douala 5,ndogbom,r a s,chretien (catholique),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911,2019-12-09,1991-04-20,secondaire,homme,célibataire,hotelier,douala 2,new bell,camerounaise,chretien (protestant ),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1912,2019-12-06,1997-01-12,secondaire,homme,célibataire,commerçant (e),douala 3,yassa,camerounaise,pas précisé,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913,2019-05-15,1991-05-05,universitaire,homme,célibataire,technicien genie civil,douala 5,bepanda,r a s,pas précisé,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_with_Bthd.to_excel("./cleaned_sheet.xlsx")